<a href="https://colab.research.google.com/github/ProsperChuks/hamoye-assessment-3/blob/main/assesstment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, confusion_matrix

In [60]:
path = '/content/drive/MyDrive/Colab Notebooks/Hamoye Intership/classification/Data_for_UCI_named.csv'
df = pd.read_csv(path)
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


checking for null values and duplicates

In [61]:
print(df.isnull().sum())

print(df.duplicated().any())

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64
False


No null values and 0 duplicates

dropping unnecessary column

In [62]:
df = df.drop(columns=['stab'])
df.head(3)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable


scaling using Standard Scaler

In [63]:
X = df.drop('stabf', axis=1)
y = df['stabf']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=1)

scaler = StandardScaler()
stand_train = scaler.fit_transform(x_train)
stand_train = pd.DataFrame(stand_train, columns=x_train.columns)

stand_test = scaler.fit_transform(x_test)
stand_test = pd.DataFrame(stand_test, columns=x_test.columns)

random forest model

In [73]:
from sklearn.ensemble import RandomForestClassifier

r_forest = RandomForestClassifier(random_state=1)
r_forest.fit(stand_train, y_train)

pred = r_forest.predict(stand_test)
pred

accuracy = accuracy_score(y_true=y_test, y_pred=pred)
print('%.4f'%accuracy)

0.9280


extra trees classifier

In [79]:
from sklearn.ensemble import ExtraTreesClassifier

ext = ExtraTreesClassifier(random_state=1)
ext.fit(stand_train, y_train)

pred = ext.predict(stand_test)
pred

accuracy = accuracy_score(y_true=y_test, y_pred=pred)
print('%.4f'%accuracy)

0.9260


In [80]:
n_estimators = [50,100,300,500,1000] 
min_samples_split = [2,3,5,7,9] 
min_samples_leaf = [1,2,4,6,8] 
max_features = ['auto','sqrt','log2',None] 
hyperparameter_grid = {'n_estimators' : n_estimators,
                       'min_samples_leaf' : min_samples_leaf,
                       'min_samples_split' : min_samples_split,
                       'max_features' : max_features }

random cv search

In [81]:
from sklearn.model_selection import RandomizedSearchCV

search = RandomizedSearchCV(estimator=ext, param_distributions=hyperparameter_grid, random_state=1)
search = search.fit(stand_train, y_train)

print(search.best_params_)

{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': None}


xgboost

In [72]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=1)
xgb.fit(stand_train, y_train)

pred = xgb.predict(stand_test)
pred

accuracy = accuracy_score(y_true=y_test, y_pred=pred)
print('%.4f'%accuracy)

0.9190


light gradient boost

In [74]:
import lightgbm as lbg

lbg = lbg.LGBMClassifier(random_state=1)
lbg.fit(stand_train, y_train)

pred = lbg.predict(stand_test)
pred

accuracy = accuracy_score(y_true=y_test, y_pred=pred)
print('%.4f'%accuracy)

0.9355


new ext class

In [83]:
ext2 = ExtraTreesClassifier(random_state=1,
                              n_estimators=1000,
                              min_samples_leaf=8,
                              max_features=None,
                              min_samples_split=2)
ext2.fit(stand_train, y_train)
pred = ext2.predict(stand_test)
pred

accuracy = accuracy_score(y_true=y_test, y_pred=pred)
print('%.4f'%accuracy)

0.9285


most and least important feature

In [93]:
sorted(zip(search.best_estimator_.feature_importances_, X))

[(0.003683422151688322, 'p1'),
 (0.004962486591192238, 'p4'),
 (0.005336864710946151, 'p2'),
 (0.005429268421191957, 'p3'),
 (0.10256244080927947, 'g1'),
 (0.10757764577478764, 'g2'),
 (0.10954089174337298, 'g4'),
 (0.11306267999167334, 'g3'),
 (0.13468028520386593, 'tau3'),
 (0.1354167630909727, 'tau4'),
 (0.13723974766109256, 'tau1'),
 (0.14050750384993677, 'tau2')]